In [1]:
# For installation / restart kernel just after running this cell
#!pip install appmode
#!jupyter nbextension     enable --py --sys-prefix appmode
#!jupyter serverextension enable --py --sys-prefix appmode

In [2]:
%gui asyncio

In [3]:
import yaml
import asyncio
from ocs_academic_hub import HubClient
import IPython
import pandas as pd
import os

In [4]:
# %env OCS_HUB_CONFIG=config-dv.ini
ask_for_secret = not bool(os.environ.get("OCS_HUB_CONFIG", False))
cheat_key = os.environ.get("OCS_HUB_CHEAT", "hubdemo2020")
# ask_for_secret

In [5]:
with open("ucdavis-building-ceed-dv.yaml", "r") as stream:
    building_ceeds = yaml.safe_load(stream)

deschutes_dvid = {f"Fermenter #{i}": {} for i in range(31, 37)}
for k in deschutes_dvid.keys():
    fv_id = k[-2:]
    for dv in ["ADF_Prediction", "Cooling_Prediction", "PCA"]:
        deschutes_dvid[k][dv] = f"HubDV_{dv}_fv{fv_id}"

UCDAVIS_DB = "UCDavis.Facilities"
DESCHUTES_DB = "Deschutes.Fermenters"


def namespace_of(db):
    if db == UCDAVIS_DB:
        return "UC__Davis"
    elif db == DESCHUTES_DB:
        return "fermenter_vessels"
    else:
        return None


building_ceeds_dvids = {UCDAVIS_DB: building_ceeds, DESCHUTES_DB: deschutes_dvid}

In [6]:
from ipywidgets import ToggleButton, DatePicker, Select, interactive, Layout, Output, Text
from IPython.display import display
import datetime

![](https://academichub.blob.core.windows.net/images/plain_blue_banner_v2.png)

Version 0.9.7

### The Academic Hub rely on [OSISoft OCS](https://www.osisoft.com/solutions/cloud/vision/), a cloud-native data infrastructure platform to collect, store and provide <br>application-ready process data for analysis, sharing and decision making. <br><br>To learn more about, click [OSIsoft Academic Hub](https://academichub.blob.core.windows.net/images/PDF%20for%20CSV%20GUI%20-%20V%201.1.pdf).
---
### Steps to download a CSV 

1. Select a Database (available databases, click for info: [UCDavis.Facilities](https://academic.osisoft.com/hackdavis), [Deschutes.Fermenters](https://www.osisoft.com/presentations/introduction-to-time-series-analysis-with-pi-system-data/))
1. Select an Asset 
2. Select a Data View (returns interpolated sensor data in tabular format, for [more info](https://ocs-docs.osisoft.com/Documentation/DataViews/Data_Views_Overview.html))
3. Pick Start and End Dates
4. Click **Download CSV** to download a result CSV with requested data

**NOTE: to download another CSV, reload this web page**

In [7]:
def read_df_count():
    with open("df_count") as f:
        i = int(f.readline())
    j = i + 1
    # print(f"i={i}  j={j}")
    with open("df_count", "w") as f:
        f.write(str(j))
    return i

In [8]:
def wait_for_change(widget, value):
    future = asyncio.Future()

    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)

    widget.observe(getvalue, value)
    return future


def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink

    i = FileLink(csv_file_name)
    # raise Exception("here1")
    if delete_prompt:
        a = input("Press enter to delete the file after you have downloaded it.")
        import os

        os.remove(csv_file_name)
    return i

In [9]:
building_ceeds_dvid = building_ceeds_dvids[UCDAVIS_DB]


def select_ceed(asset):
    ceedW.options = building_ceeds_dvid[asset].keys()
    print(f"Current asset: {asset}")


k = ToggleButton(
    value=False,
    description="Download CSV",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Description",
    icon="check",
)

init_start = datetime.date(2017, 3, 17)
init_end = datetime.date(2017, 9, 17)
start = DatePicker(description="Start Date", disabled=False, value=init_start)
end = DatePicker(description="End Date", disabled=False, value=init_end)
secret = Text(
    value="", placeholder="hub secret key", description="Secret:", disabled=False
)

databaseW = Select(options=[UCDAVIS_DB, DESCHUTES_DB], layout=Layout(height="60px"))
db_init = databaseW.value


def select_asset(database):
    global building_ceeds_dvid
    building_ceeds_dvid = building_ceeds_dvids[database]
    buildingW.options = building_ceeds_dvid.keys()


buildingW = Select(
    options=building_ceeds_dvid.keys(), layout=Layout(width="40%", height="180px")
)
init = buildingW.value
ceedW = Select(options=building_ceeds_dvid[init])


def print_ceed(dataview):
    print(f"Current dataview: {dataview}")


from IPython.display import FileLink


df_count = read_df_count()
df_filename = f"./dataview-result-{df_count}.csv"
os.system(f"touch {df_filename}")
fl = FileLink(df_filename)


def display_df_link():

    return


j = interactive(print_ceed, dataview=ceedW)
i = interactive(select_ceed, asset=buildingW)
l = interactive(select_asset, database=databaseW)
# fi = display_df_link()  # interactive(display_link, link=df_count)


out = Output()


def my_append_stdout(message, end="\n"):
    out.append_stdout(message + end)


async def f():
    out.append_stdout(
        "Click *Download CSV* button to get result CSV data with current parameters\n"
    )
    x = await wait_for_change(k, "value")
    out.append_stdout(
        f"[ {buildingW.value} | {ceedW.value} | {start.value} | {end.value} | {building_ceeds_dvid[buildingW.value][ceedW.value]} ]\n"
    )
    out.append_stdout(f"Working on getting data from OCS...\n")

    with out:
        if True: # not ask_for_secret:  # os.environ.get("OCS_HUB_CONFIG", False):
            hub_client = HubClient()
        else:
            if len(secret.value) != 44 and secret.value != cheat_key:
                out.append_stdout(
                    f"@@@@ key missing or incorrect length (must be 44, got {len(secret.value)})\n"
                )
                out.append_stdout(f"@@@@ refresh this page to start over\n")
                out.append_stdout(
                    f"@@@@ --------------------------------------------------------\n"
                )
                return
            secret_key = (
                "eLun6/9tvplOM4wtOV0G4moA7u2T0NNB8N28y0UNesc="
                if secret.value == cheat_key
                else secret.value
            )

            config = f"config-{df_count}.ini"
            os.environ["OCS_HUB_CONFIG"] = config
            with open(config, "w") as f:
                f.write(
                    f"""
; IMPORTANT: replace these values with those provided by OSIsoft
[Access]
Resource = https://dat-b.osisoft.com
Tenant = 65292b6c-ec16-414a-b583-ce7ae04046d4
ApiVersion = v1

[Credentials]
ClientId = b6cd8dd9-eb08-475e-ac51-fbdc76fdc8f2
ClientSecret = {secret_key}
"""
                )
            hub_client = HubClient()
        df = hub_client.dataview_interpolated_pd(
            namespace_of(databaseW.value),
            building_ceeds_dvid[buildingW.value][ceedW.value],
            str(start.value),
            str(end.value),
            interval="00:10:00",
        )
    # out.append_stdout(f"df={df}\n")
    out.append_stdout(f"OK to click link below to download CSV \n")
    try:
        fl = display(
            csv_download_link(
                df,
                f"{os.getcwd()}/dataview-result-{df_count}.csv",  # {building_ceeds_dvid[buildingW.value][ceedW.value]}
                delete_prompt=False,
            )
        )
    except Exception as e:
        out.append_stdout(f"@@@ exc={e}\n")

    out.append_stdout(f"Download status OK\n")


asyncio.ensure_future(f())

display(l)
display(i)
display(j)
display(start)
display(end)
# if ask_for_secret:
#    display(secret)
display(k)
display(out)
display(fl)

interactive(children=(Select(description='database', layout=Layout(height='60px'), options=('UCDavis.Facilitie…

interactive(children=(Select(description='asset', layout=Layout(height='180px', width='40%'), options=('ARC Pa…

interactive(children=(Select(description='dataview', options=('ChilledWater_MBTU', 'Electricity', 'Steam'), va…

DatePicker(value=datetime.date(2017, 3, 17), description='Start Date')

DatePicker(value=datetime.date(2017, 9, 17), description='End Date')

ToggleButton(value=False, description='Download CSV', icon='check', tooltip='Description')

Output()

C:\Users\cfoisy\Documents\Hub\experimental\csv-omf\appmode\dataview-result-3.csv